In [1]:
import pandas as pd
import csv
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/la_airbnb/{}.csv'

la_file = file_url.format('listings_details')
reviews_file = file_url.format('reviews_details')
calendar_file = file_url.format('calendar_details')

In [3]:
file_url = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

sf_file = file_url.format('sf_airbnb/listings_details')
ok_file = file_url.format('ok_airbnb/listings_details')
sc_file = file_url.format('sc_airbnb/listings_details')
scz_file = file_url.format('scz_airbnb/listings_details')
pg_file = file_url.format('pg_airbnb/listings_details')
sd_file = file_url.format('sd_airbnb/listings_details')
la_file = file_url.format('la_airbnb/listings_details')

In [4]:
listings_sf = pd.read_csv(sf_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_ok = pd.read_csv(ok_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_sc = pd.read_csv(sc_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_scz = pd.read_csv(scz_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_sd = pd.read_csv(sd_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])
listings_la = pd.read_csv(la_file, parse_dates=['last_scraped','host_since', 'first_review', 'last_review'])

In [5]:
print (listings_sf.shape, listings_ok.shape, listings_sc.shape, listings_scz.shape, listings_sd.shape,listings_la.shape)

(6633, 96) (2898, 96) (5668, 96) (1570, 96) (11726, 96) (42337, 96)


In [6]:
frames = [listings_sf, listings_ok, listings_sc, listings_scz, listings_sd, listings_la]

In [7]:
listings = pd.concat(frames)

In [8]:
listings.columns.values

array(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name',
       'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'access',
       'interaction', 'house_rules', 'thumbnail_url', 'medium_url',
       'picture_url', 'xl_picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode',
       'market', 'smart_location', 'country_code', 'country', 'latitude',
       'longitude', 'is_location_exact', 'property_type', 'room_type',
       'accommodates', 'bath

In [9]:
df = listings[["host_response_rate", "host_acceptance_rate", "host_is_superhost",
               "host_listings_count", "zipcode", "property_type","room_type", "accommodates", "bathrooms", "bedrooms", 
               "beds", "price", "number_of_reviews", "review_scores_rating", "cancellation_policy", 
               "reviews_per_month"]]

In [10]:
df.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating,cancellation_policy,reviews_per_month
0,92%,NaN,t,1.0,94117,Apartment,Entire home/apt,3,1.0,1.0,2.0,$170.00,152,97.0,moderate,1.38
1,100%,NaN,f,2.0,94110,Apartment,Entire home/apt,5,1.0,2.0,3.0,$235.00,112,98.0,strict_14_with_grace_period,0.99
2,100%,NaN,f,10.0,94117,Apartment,Private room,2,4.0,1.0,1.0,$65.00,17,85.0,strict_14_with_grace_period,0.16
3,100%,NaN,f,10.0,94117,Apartment,Private room,2,4.0,1.0,1.0,$65.00,7,91.0,strict_14_with_grace_period,0.15
4,100%,NaN,f,2.0,94117,House,Entire home/apt,5,1.5,2.0,2.0,$675.00,26,97.0,strict_14_with_grace_period,0.24


In [11]:
# % of NaN values
(len(df)-df.count())/len(df)*100

host_response_rate       16.222894
host_acceptance_rate    100.000000
host_is_superhost         0.016941
host_listings_count       0.016941
zipcode                   1.473910
property_type             0.000000
room_type                 0.000000
accommodates              0.000000
bathrooms                 0.110120
bedrooms                  0.040942
beds                      0.090355
price                     0.000000
number_of_reviews         0.000000
review_scores_rating     20.983454
cancellation_policy       0.000000
reviews_per_month        19.708606
dtype: float64

In [12]:
df = df.drop(['host_acceptance_rate'], axis=1)

In [13]:
# drop NaN rows
df2=df.dropna(axis=0)

In [14]:
# % of rows droppped
(len(df)-len(df2))/len(df)*100

30.174215044047887

In [15]:
df2.shape

(49459, 15)

In [16]:
pd.options.mode.chained_assignment = None  # default='warn'

df2['host_response_rate'] = df2['host_response_rate'].astype(str)
df2['price'] = df2['price'].astype(str)

In [17]:
# clean data
pd.options.mode.chained_assignment = None  # default='warn'

df2['price'] = df2['price'].str.replace("[$, ]", "").astype("float")
df2['host_response_rate'] = df2['host_response_rate'].str.replace("%", "").astype("float")

In [18]:
df2['superhost']=np.where(df2['host_is_superhost']=='t',1,0)
del df2['host_is_superhost']

In [19]:
# select non-numeric variables and create dummies
non_num_vars = df2.select_dtypes(include=['object']).columns
df2[non_num_vars].head()

,zipcode,property_type,room_type,cancellation_policy
0,94117,Apartment,Entire home/apt,moderate
1,94110,Apartment,Entire home/apt,strict_14_with_grace_period
2,94117,Apartment,Private room,strict_14_with_grace_period
3,94117,Apartment,Private room,strict_14_with_grace_period
4,94117,House,Entire home/apt,strict_14_with_grace_period


In [20]:
dummy_vars = pd.get_dummies(df2[non_num_vars])

In [21]:
dummy_vars.head()

,zipcode_94014.0,zipcode_94102.0,zipcode_94103.0,zipcode_94104.0,zipcode_94105.0,zipcode_94106.0,zipcode_94107.0,zipcode_94108.0,zipcode_94109.0,zipcode_94110.0,...,property_type_Yurt,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [22]:
# drop non-numeric variables from df2 and add the dummies
df3=df2.drop(non_num_vars,axis=1)
df3 = pd.merge(df3,dummy_vars, left_index=True, right_index=True)
df3.head()

,host_response_rate,host_listings_count,accommodates,bathrooms,bedrooms,beds,price,number_of_reviews,review_scores_rating,reviews_per_month,...,property_type_Yurt,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60
0,92.0,1.0,3,1.0,1.0,2.0,170.0,152,97.0,1.38,...,0,1,0,0,0,1,0,0,0,0
0,92.0,1.0,3,1.0,1.0,2.0,170.0,152,97.0,1.38,...,0,0,1,0,0,0,0,1,0,0
0,92.0,1.0,3,1.0,1.0,2.0,170.0,152,97.0,1.38,...,0,0,1,0,0,1,0,0,0,0
0,92.0,1.0,3,1.0,1.0,2.0,170.0,152,97.0,1.38,...,0,1,0,0,0,0,0,1,0,0
0,100.0,6.0,2,1.0,1.0,1.0,65.0,25,91.0,0.51,...,0,1,0,0,0,1,0,0,0,0


In [23]:
# split into test and training data
np.random.seed(1)
indices = np.random.permutation(len(df3))
train_size = int(round(0.8*len(df3)))
test_size = len(df3)-train_size

y = df3['price']
x = df3.drop('price', axis =1)

x.train = x.iloc[indices[0:train_size]]
y.train = y.iloc[indices[0:train_size]]
x.test = x.iloc[indices[train_size+1:]]
y.test = y.iloc[indices[train_size+1:]]

x2 = x.train.as_matrix()
y2 = y.train.as_matrix()

/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if sys.path[0] == '':
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app
/Users/xzhou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [24]:
import statsmodels.api as sm
olsmod = sm.OLS(y2,x2)
olsres = olsmod.fit()
print(olsres.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.401
Model:                            OLS   Adj. R-squared:                  0.397
Method:                 Least Squares   F-statistic:                     116.4
Date:                Thu, 13 Sep 2018   Prob (F-statistic):               0.00
Time:                        14:15:00   Log-Likelihood:            -6.2110e+05
No. Observations:               94161   AIC:                         1.243e+06
Df Residuals:                   93622   BIC:                         1.248e+06
Df Model:                         538                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.2225      0.053     -4.224      0.0

In [25]:
# different method
from sklearn import linear_model
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state=1)

In [26]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_estimators=200, 
                               criterion='mse', 
                               random_state=3, 
                               n_jobs=-1)
forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 3062.093, test: 18534.614
R^2 train: 0.943, test: 0.635
